In [ ]:
!pip install tensorflow==2.12
import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()

In [ ]:
!pip install keras-video-generators==1.0.11

In [ ]:
import os
import glob
from tensorflow import keras
import keras_video
from keras_video import VideoFrameGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Flatten, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import VGG19

In [ ]:
test_dataset_path = '/kaggle/input/cric-wicket-classification-dataset/test/'
train_dataset_path = '/kaggle/input/cric-wicket-classification-dataset/train/'

In [ ]:
train_glob_pattern= train_dataset_path +'{classname}/*'
valid_glob_pattern= test_dataset_path + '{classname}/*'

In [ ]:
train_classes = [os.path.basename(i) for i in glob.glob(train_dataset_path + '*')]
train_classes.sort()

print(train_classes)

valid_classes = [os.path.basename(i) for i in glob.glob(test_dataset_path + '*')]
valid_classes.sort()

print(valid_classes)

In [ ]:
SIZE = (224, 224)
CHANNELS = 3
NBFRAME = 11
BS = 4

In [ ]:
train_data_aug = keras.preprocessing.image.ImageDataGenerator(
        dtype = 'float16',
        rescale = 1./255,
        zoom_range=.1,
        horizontal_flip=True,
        rotation_range=15
    )

train = VideoFrameGenerator(
    classes=train_classes,
    glob_pattern=train_glob_pattern,
    nb_frames=NBFRAME,
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=train_data_aug,
    use_frame_cache=True)

In [ ]:
valid_data_aug = keras.preprocessing.image.ImageDataGenerator(
        dtype = 'float16',
        rescale = 1./255
    )

valid = VideoFrameGenerator(
    classes=valid_classes,
    glob_pattern=valid_glob_pattern,
    nb_frames=NBFRAME,
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=valid_data_aug,
    use_frame_cache=True)

In [ ]:
def action_model(shape=(11, 224, 224, 3), nbout=4):
    convnet = VGG19(weights='imagenet', include_top=False, input_shape=shape[1:])
    convnet.summary()

    # enable if you want to enable fine-tuning
    trainable = 8  # change this value
    for layer in convnet.layers[:-trainable]:
        layer.trainable = False
    for layer in convnet.layers[-trainable:]:
        layer.trainable = True

    # enable if you don't want fine-tuning
    # for layer in convnet.layers:
    #     layer.trainable = False

    model = keras.Sequential()
    model.add(TimeDistributed(convnet, input_shape=shape))
    model.add(TimeDistributed(Flatten()))

    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Dropout(0.5))

    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.5))

    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nbout, activation='softmax'))

    model.summary()
    return model

In [ ]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (15, 180, 180, 3)
model = action_model(INSHAPE, len(train_classes))
optimizer = Adam(learning_rate=1e-5)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS=100
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('/kaggle/working/adam_VGG_LSTM_cric4_learning_1e-5_.h5',
                                       monitor='val_accuracy',
                                       verbose=1,
                                       save_best_only=True,
                                       mode='max',
                                       save_freq='epoch'),
                                       EarlyStopping(monitor='val_accuracy', patience=10, mode="max", restore_best_weights=True)
                                       ]

In [ ]:
history = model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.figure(figsize=(8, 6))
sns.lineplot(x=range(1, len(history.history["accuracy"]) + 1), y=history.history["accuracy"], label='train acc')
sns.lineplot(x=range(1, len(history.history["val_accuracy"]) + 1), y=history.history["val_accuracy"], label='test acc')
plt.title('Model Performance: Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='lower right')
plt.show()


plt.figure(figsize=(8, 6))
sns.lineplot(x=range(1, len(history.history["loss"]) + 1), y=history.history["loss"], label='train loss')
sns.lineplot(x=range(1, len(history.history["val_loss"]) + 1), y=history.history["val_loss"], label='test loss')
plt.title('Model Performance: Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
import pandas as pd
hist_df = pd.DataFrame(history.history)
hist_csv_file = '/kaggle/working/adam_LSTM_cric4_learning_1e-5_.csv'

with open(hist_csv_file, mode='w') as f:
     hist_df.to_csv(f)

In [ ]:
import numpy as np

batch_size = BS
steps_per_epoch = len(valid)

y_true = np.empty(0, dtype=int)
y_pred = np.empty(0, dtype=int)

for i in range(steps_per_epoch):
    x_batch, y_batch = next(valid)
    y_true_batch = np.argmax(y_batch, axis=1)
    y_pred_batch = np.argmax(model.predict(x_batch), axis=1)
    y_true = np.concatenate([y_true, y_true_batch])
    y_pred = np.concatenate([y_pred, y_pred_batch])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print('Confusion Matrix')
conf_matrix = confusion_matrix(y_true, y_pred)
print(conf_matrix)

In [ ]:
plt.figure(figsize=(6,4), dpi=100)

sns.set(font_scale = 1.1)

ax = sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='mako')

ax.set_xlabel("Predicted Label", fontsize=15, labelpad=20, weight='bold')
ax.xaxis.set_ticklabels(['WK Catch', 'Bowled Out', 'Slip Catch', 'LBW'], weight='bold', rotation=90)


ax.set_ylabel("True Label", fontsize=15, labelpad=20,weight='bold')
ax.yaxis.set_ticklabels(['WK Catch', 'Bowled Out', 'Slip Catch', 'LBW'], weight='bold', rotation=0)

ax.set_title("Confusion Matrix", fontsize=18, pad=20)

plt.show()

In [ ]:
print('Classification Report')
target_names = ['WK Catch', 'Bowled Out', 'Slip Catch', 'LBW']

target_names = np.asarray(target_names)
print(classification_report(y_true, y_pred, target_names=target_names))